In [1]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext autoreload
%autoreload 1
from imports import *
from functions import *

Connection to MySQL

In [3]:
mysql_user = "flosrv"
password = "Nesrine123"
host = "localhost"
port = 3306
database = "Oceanography_data_analysis"
metadata = MetaData()
# Connect to the database
engine = create_engine(f"mysql+mysqlconnector://{mysql_user}:{password}@{host}/{database}", isolation_level ='AUTOCOMMIT')

Get Available Stations ID List

In [5]:
# get all stations and some metadata as a Pandas DataFrame
stations_df = api.stations()
# parse the response as a dictionary
stations_df = api.stations(as_df=True)

print(len(stations_df))

146


Filter Buoys by Remarks

In [6]:
access_error_url_list = []

# Liste de mots à rechercher dans la colonne "Remark"
blacklist = ["Failure", "ceased", "failed", "recovered", "stopped", 'adrift']
stations_id_set = set()

print(f'Avant Filtre: {stations_df.shape[0]}')

# Liste pour collecter les indices à supprimer
indices_a_supprimer = []

# Parcours des lignes de la DataFrame
for idx, row in stations_df.iterrows():
    station_id = row["Station"]
    station_Location = row["Hull No./Config and Location"]  # Extraire la valeur de la cellule pour chaque ligne
    
    # Extraction du nom de la station si un ")" est trouvé
    if ")" in station_Location:
        station_name = station_Location.split(')')[1].rstrip(" )")  # On enlève l'espace et la parenthèse en fin de chaîne
    else:
        station_name = station_Location.strip()  # Si pas de ")", on garde toute la chaîne

    station_name = station_name.rstrip(" )").replace("(", "").replace(")", "").strip()

    # Nettoyage final pour enlever toute parenthèse ou espace en fin de nom
    station_name = station_name.rstrip(" )")

    # Vérifier si "Remark" n'est pas NaN et si un des éléments de blacklist est dans "Remark"
    if isinstance(row["Remark"], str) and any(blacklist_word.lower() in row["Remark"].lower() for blacklist_word in blacklist):
        # Ajouter l'index à la liste
        indices_a_supprimer.append(idx)
        print(f'Buoy {station_id} Failed the Remark Test !')
        url = get_buoy_url(station_id)
        access_error_url_list.append(url)
    else:
        print(f'Buoy {station_id} Passed the Remark Test !')

# Supprimer les lignes après la boucle
stations_df.drop(index=indices_a_supprimer, inplace=True)

print(f'Après Filtre: {stations_df.shape[0]}')

Avant Filtre: 146
Buoy 41001 Failed the Remark Test !
Buoy 41002 Failed the Remark Test !
Buoy 41004 Passed the Remark Test !
Buoy 41008 Passed the Remark Test !
Buoy 41009 Failed the Remark Test !
Buoy 41010 Passed the Remark Test !
Buoy 41013 Passed the Remark Test !
Buoy 41025 Failed the Remark Test !
Buoy 41040 Passed the Remark Test !
Buoy 41041 Failed the Remark Test !
Buoy 41043 Passed the Remark Test !
Buoy 41044 Passed the Remark Test !
Buoy 41046 Failed the Remark Test !
Buoy 41047 Failed the Remark Test !
Buoy 41048 Failed the Remark Test !
Buoy 41049 Passed the Remark Test !
Buoy 42001 Passed the Remark Test !
Buoy 42002 Passed the Remark Test !
Buoy 42003 Passed the Remark Test !
Buoy 42012 Passed the Remark Test !
Buoy 42019 Passed the Remark Test !
Buoy 42020 Passed the Remark Test !
Buoy 42035 Passed the Remark Test !
Buoy 42036 Passed the Remark Test !
Buoy 42039 Failed the Remark Test !
Buoy 42040 Failed the Remark Test !
Buoy 42055 Failed the Remark Test !
Buoy 42056

Build Buoys_datas Dict

In [7]:
# Dictionnaire pour stocker les DataFrames, clé : ID de la bouée, valeur : DataFrame
buoy_datas = {}
buoy_list = []

# Parcours de chaque bouée dans stations_df
for index, row in stations_df.iterrows():
    buoy_id = row['Station']
    metadata = get_station_metadata(buoy_id)

    # ✅ Récupérer les données sous forme de dictionnaire
    buoy_info = parse_buoy_json(metadata)

    # ✅ Stocker directement les données dans buoy_datas
    buoy_datas[buoy_id] = buoy_info
    buoy_list.append(buoy_id)

# Affichage du nombre de bouées réussies et échouées
print(f"Nombre de bouées traitées : {len(buoy_datas)}\n")

# Afficher le contenu de buoy_datas

first_key =next(iter(buoy_datas))
first_key
buoy_datas[first_key]


🔍 Début du parsing de la bouée...
🌍 Zone de la station : grays reef
🆔 Station ID : 41008
✅ Coordonnées extraites : Latitude = 31.40N, Longitude = 80.87W
🌊 Water Depth : 16 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation : 2.4
💨 Anemometer Height : 3.8
🌤️ Air Temp Height : 3.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=41008
✅ Parsing terminé !


🔍 Début du parsing de la bouée...
🌍 Zone de la station : ne st martin
🆔 Station ID : 41044
✅ Coordonnées extraites : Latitude = 21.58N, Longitude = 58.63W
🌊 Water Depth : 5419 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation : 2.4
💨 Anemometer Height : 3.8
🌤️ Air Temp Height : 3.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=41044
✅ Parsing terminé !


🔍 Début du parsing de la bouée...
🌍 Zone de la station : south bermuda
🆔 Station ID : 41049
✅ Coordonnées extraites : Latitude = 27.50N, Longitude = 62.27W
🌊 Water Depth : 5480 m
🌡️ Sea Temp Depth : 1.5
🌬️ Barometer Elevation : 2.7
💨 Anemometer 

{'station_zone': 'grays reef',
 'lat_buoy': '31.40N',
 'lon_buoy': '80.87W',
 'Water_depth': '16 m',
 'sea_temp_depth': '2',
 'Barometer_elevation': '2.4',
 'Anemometer_height': '3.8',
 'Air_temp_height': '3.4',
 'url': 'https://www.ndbc.noaa.gov/station_page.php?station=41008'}

In [8]:
def create_table_in_mysql(df: pd.DataFrame, table_name: str, engine):
    # Vérifier si la table existe déjà
    check_table_sql = f"SHOW TABLES LIKE '{table_name}';"

    try:
        with engine.connect() as connection:
            result = connection.execute(text(check_table_sql))
            if result.fetchone():
                print(f"⚠️ La table '{table_name}' existe déjà dans MySQL.")
                return  # Ne pas recréer la table si elle existe déjà

            # Définition des colonnes
            column_definitions = []
            for column_name, dtype in df.dtypes.items():
                if dtype == 'object': 
                    column_type = "VARCHAR(255)"
                elif dtype == 'int64':  
                    column_type = "INT"
                elif dtype == 'float64': 
                    column_type = "FLOAT"
                elif dtype == 'datetime64[ns]': 
                    column_type = "DATETIME"
                elif dtype == 'datetime64[ns, UTC]': 
                    column_type = "DATETIME"
                elif dtype == 'timedelta64[ns]':  
                    column_type = "TIME"
                elif dtype == 'bool':  
                    column_type = "BOOLEAN"
                elif dtype == 'time64[ns]': 
                    column_type = "TIME"
                else:
                    column_type = "VARCHAR(255)" 

                column_definitions.append(f"`{column_name}` {column_type}")

            # Création de la table
            create_table_sql = f"CREATE TABLE `{table_name}` ({', '.join(column_definitions)});"
            connection.execute(text(create_table_sql))
            print(f"✅ Table '{table_name}' créée avec succès dans MySQL.")

    except Exception as e:
        print(f"❌ Erreur lors de la création de la table : {e}")


In [ ]:
def insert_new_rows(engine, df: pd.DataFrame, table_name: str):
    """
    Insère uniquement les nouvelles lignes de df dans MySQL en comparant la colonne 'Datetime'.
    """
    try:
        # Vérifier si la table existe et récupérer la valeur max de Datetime
        query = f"SELECT MAX(Datetime) FROM `{table_name}`;"
        with engine.connect() as connection:
            result = connection.execute(text(query)).scalar()  # Récupérer la valeur max
        
        # Si la table est vide, insérer tout le DataFrame
        if result is None:
            print(f"💾 Aucune donnée dans '{table_name}', insertion de toutes les lignes.")
            df.to_sql(name=table_name, con=engine, if_exists='append', index=False)
            print(f"✅ {len(df)} nouvelles lignes insérées dans '{table_name}'.")
            return
        
        # Filtrer les nouvelles lignes (celles avec un Datetime plus grand que la valeur max en base)
        df["Datetime"] = pd.to_datetime(df["Datetime"])  # S'assurer que la colonne est bien en datetime
        new_rows = df[df["Datetime"] > result]

        if new_rows.empty:
            print(f"✅ Aucune nouvelle ligne à insérer dans '{table_name}'.")
            return

        # Insérer uniquement les nouvelles lignes
        new_rows.to_sql(name=table_name, con=engine, if_exists='append', index=False)
        print(f"✅ {len(new_rows)} nouvelles lignes insérées dans '{table_name}'.")

    except Exception as e:
        print(f"❌ Erreur lors de l'insertion des nouvelles lignes : {e}")


In [20]:
# 🚀 Démarrage du processus
print("\n🚀 Démarrage du processus de collecte et d'insertion des données...\n")

# Initialisation des compteurs
marine_data_collected_successfully = marine_data_collected_failed = 0
marine_data_inserted_successfully = marine_data_inserted_failed = 0
meteo_data_collected_successfully = meteo_data_collected_failed = 0
meteo_data_inserted_successfully = meteo_data_inserted_failed = 0

success = False
total_stations = stations_df.shape[0]
count = 0

# 🔄 Parcours des bouées / stations
for idx, row in stations_df.iterrows():
    buoy_id = row["Station"]

    ######### 🌊 MARINE DATA #########
    try:
        df_marine = NDBC.realtime_observations(buoy_id)
        if df_marine is None or df_marine.empty:
            marine_data_collected_failed += 1
            continue

        marine_data_collected_successfully += 1
    except Exception as e:
        print(f"⚠️ Erreur collecte marine {buoy_id}: {e}")
        marine_data_collected_failed += 1
        continue

    # Ajout des métadonnées
    try:
        buoy_info = buoy_datas.get(buoy_id, {})
        Lat, Lon = buoy_info.get('lat_buoy'), buoy_info.get('lon_buoy')
        if Lat is None or Lon is None:
            raise ValueError(f"Données manquantes pour {buoy_id}")

        df_marine['Lat'] = Lat
        df_marine['Lon'] = Lon
        df_marine['Water_depth'] = buoy_info.get('Water_depth', None)
        df_marine.columns = ['Datetime' if 'date' in col.lower() or 'time' in col.lower() else col for col in df_marine.columns]

        station_zone = safe_get(parse_buoy_json(get_station_metadata(buoy_id)), "station_zone")
        Bronze_Marine_table_Name = f"bronze_station_{buoy_id}_marine_{station_zone}".replace('.', '_').replace('-', '_')

        print(f"✅ Données marines collectées {buoy_id}")
    except Exception as e:
        print(f"⚠️ Erreur métadonnées marine {buoy_id}: {e}")
        marine_data_collected_failed += 1
        continue

    # Création de la table et insertion des nouvelles lignes
    try:
        create_table_in_mysql(engine=engine, df=df_marine, table_name=Bronze_Marine_table_Name)
        insert_new_rows(engine=engine, df=df_marine, table_name=Bronze_Marine_table_Name)
        marine_data_inserted_successfully += 1
    except Exception as e:
        print(f"❌💾 Erreur insertion marine {buoy_id}: {e}")
        marine_data_inserted_failed += 1
        continue

    ######### ⛅ METEO DATA #########
    try:
        df_meteo = meteo_api_request([Lat, Lon])
        if df_meteo is None or df_meteo.empty:
            meteo_data_collected_failed += 1
            continue

        df_meteo.columns = ['Datetime' if 'date' in col.lower() or 'time' in col.lower() else col for col in df_meteo.columns]
        meteo_data_collected_successfully += 1
    except Exception as e:
        print(f"⚠️ Erreur collecte météo {buoy_id}: {e}")
        meteo_data_collected_failed += 1
        continue

    # Création de la table et insertion des nouvelles lignes
    try:
        Bronze_Meteo_table_Name = f"bronze_station_{buoy_id}_meteo_{station_zone}".replace('.', '_').replace('-', '_')
        create_table_in_mysql(engine=engine, df=df_meteo, table_name=Bronze_Meteo_table_Name)
        insert_new_rows(engine=engine, df=df_meteo, table_name=Bronze_Meteo_table_Name)
        meteo_data_inserted_successfully += 1
    except Exception as e:
        print(f"❌💾 Erreur insertion météo {buoy_id}: {e}")
        meteo_data_inserted_failed += 1
        continue

    count += 1
    if count == total_stations:
        success = True

# 🔚 Résumé final
if success:
    print("\n📝 Résumé final :")
    print(f"🌊 Marine - Collecte ✅ {marine_data_collected_successfully} ❌ {marine_data_collected_failed}")
    print(f"🌊 Marine - Insertion ✅ {marine_data_inserted_successfully} ❌ {marine_data_inserted_failed}")
    print(f"⛅ Météo - Collecte ✅ {meteo_data_collected_successfully} ❌ {meteo_data_collected_failed}")
    print(f"⛅ Météo - Insertion ✅ {meteo_data_inserted_successfully} ❌ {meteo_data_inserted_failed}")



🚀 Démarrage du processus de collecte et d'insertion des données...


🔍 Début du parsing de la bouée...
🌍 Zone de la station : grays reef
🆔 Station ID : 41008
✅ Coordonnées extraites : Latitude = 31.40N, Longitude = 80.87W
🌊 Water Depth : 16 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation : 2.4
💨 Anemometer Height : 3.8
🌤️ Air Temp Height : 3.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=41008
✅ Parsing terminé !

📊 station_zone : grays reef
✅ Données marines collectées 41008
⚠️ La table 'bronze_station_41008_marine_grays reef' existe déjà dans MySQL.
❌ Erreur lors de l'insertion des nouvelles lignes : Invalid comparison between dtype=datetime64[ns, UTC] and datetime
✅ Table 'bronze_station_41008_meteo_grays reef' créée avec succès dans MySQL.
⚠️ Aucune donnée dans 'bronze_station_41008_meteo_grays reef', insertion de toutes les lignes.
✅ 2376 nouvelles lignes insérées dans 'bronze_station_41008_meteo_grays reef'.

🔍 Début du parsing de la bouée...
🌍 Zone de l

C:\Users\f.gionnane\AppData\Local\Temp\ipykernel_6036\1306807727.py:14: UserWarning: The provided table name 'bronze_station_BURL1_meteo_southwest pass, la' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(name=table_name, con=engine, if_exists='append', index=False)


✅ 2376 nouvelles lignes insérées dans 'bronze_station_BURL1_meteo_southwest pass, la'.

🔍 Début du parsing de la bouée...
🌍 Zone de la station : five fingers, ak
🆔 Station ID : FFIA2
✅ Coordonnées extraites : Latitude = 57.27N, Longitude = 133.63W
🌊 Water Depth : N/A
🌡️ Sea Temp Depth : None
🌬️ Barometer Elevation : 21.3
💨 Anemometer Height : 22
🌤️ Air Temp Height : 4.3
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=FFIA2
✅ Parsing terminé !

📊 station_zone : five fingers, ak
✅ Données marines collectées FFIA2
⚠️ La table 'bronze_station_FFIA2_marine_five fingers, ak' existe déjà dans MySQL.
❌ Erreur lors de l'insertion des nouvelles lignes : Invalid comparison between dtype=datetime64[ns, UTC] and datetime
✅ Table 'bronze_station_FFIA2_meteo_five fingers, ak' créée avec succès dans MySQL.
⚠️ Aucune donnée dans 'bronze_station_FFIA2_meteo_five fingers, ak', insertion de toutes les lignes.


C:\Users\f.gionnane\AppData\Local\Temp\ipykernel_6036\1306807727.py:14: UserWarning: The provided table name 'bronze_station_FFIA2_meteo_five fingers, ak' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(name=table_name, con=engine, if_exists='append', index=False)


✅ 2376 nouvelles lignes insérées dans 'bronze_station_FFIA2_meteo_five fingers, ak'.
⚠️ Erreur collecte marine KTNF1: Error accessing https://www.ndbc.noaa.gov/data/realtime2/KTNF1.txt
Server Error ( 404: Not Found)

🔍 Début du parsing de la bouée...
🌍 Zone de la station : long key, fl
🆔 Station ID : LONF1
✅ Coordonnées extraites : Latitude = 24.84N, Longitude = 80.86W
🌊 Water Depth : N/A
🌡️ Sea Temp Depth : None
🌬️ Barometer Elevation : 5.03
💨 Anemometer Height : 6.34
🌤️ Air Temp Height : 6.1
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=LONF1
✅ Parsing terminé !

📊 station_zone : long key, fl
✅ Données marines collectées LONF1
⚠️ La table 'bronze_station_LONF1_marine_long key, fl' existe déjà dans MySQL.
❌ Erreur lors de l'insertion des nouvelles lignes : Invalid comparison between dtype=datetime64[ns, UTC] and datetime
✅ Table 'bronze_station_LONF1_meteo_long key, fl' créée avec succès dans MySQL.
⚠️ Aucune donnée dans 'bronze_station_LONF1_meteo_long key, f

...


🔍 Début du parsing de la bouée...
🌍 Zone de la station : sand key, fl
🆔 Station ID : SANF1
✅ Coordonnées extraites : Latitude = 24.46N, Longitude = 81.88W
🌊 Water Depth : N/A
🌡️ Sea Temp Depth : None
🌬️ Barometer Elevation : 13.6
💨 Anemometer Height : 14.8
🌤️ Air Temp Height : 14.6
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=SANF1
✅ Parsing terminé !

📊 station_zone : sand key, fl
✅ Données marines collectées SANF1
⚠️ La table 'bronze_station_SANF1_marine_sand key, fl' existe déjà dans MySQL.
❌ Erreur lors de l'insertion des nouvelles lignes : Invalid comparison between dtype=datetime64[ns, UTC] and datetime
✅ Table 'bronze_station_SANF1_meteo_sand key, fl' créée avec succès dans MySQL.
⚠️ Aucune donnée dans 'bronze_station_SANF1_meteo_sand key, fl', insertion de toutes les lignes.
✅ 2376 nouvelles lignes insérées dans 'bronze_station_SANF1_meteo_sand key, fl'.


C:\Users\f.gionnane\AppData\Local\Temp\ipykernel_6036\1306807727.py:14: UserWarning: The provided table name 'bronze_station_SANF1_meteo_sand key, fl' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(name=table_name, con=engine, if_exists='append', index=False)



🔍 Début du parsing de la bouée...
🌍 Zone de la station : south bass island, oh
🆔 Station ID : SBIO1
✅ Coordonnées extraites : Latitude = 41.63N, Longitude = 82.84W
🌊 Water Depth : N/A
🌡️ Sea Temp Depth : None
🌬️ Barometer Elevation : 178.4
💨 Anemometer Height : 24.3
🌤️ Air Temp Height : 23.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=SBIO1
✅ Parsing terminé !

📊 station_zone : south bass island, oh
✅ Données marines collectées SBIO1
⚠️ La table 'bronze_station_SBIO1_marine_south bass island, oh' existe déjà dans MySQL.
❌ Erreur lors de l'insertion des nouvelles lignes : Invalid comparison between dtype=datetime64[ns, UTC] and datetime
✅ Table 'bronze_station_SBIO1_meteo_south bass island, oh' créée avec succès dans MySQL.
⚠️ Aucune donnée dans 'bronze_station_SBIO1_meteo_south bass island, oh', insertion de toutes les lignes.
✅ 2376 nouvelles lignes insérées dans 'bronze_station_SBIO1_meteo_south bass island, oh'.


C:\Users\f.gionnane\AppData\Local\Temp\ipykernel_6036\1306807727.py:14: UserWarning: The provided table name 'bronze_station_SBIO1_meteo_south bass island, oh' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(name=table_name, con=engine, if_exists='append', index=False)


⚠️ Erreur collecte marine SMKF1: Error accessing https://www.ndbc.noaa.gov/data/realtime2/SMKF1.txt
Server Error ( 404: Not Found)


In [11]:
# # Convertir le dictionnaire en DataFrame, en utilisant 'buoy_id' comme index
# df_buoy_datas = pd.DataFrame.from_dict(buoy_datas, orient='index')

# # Réinitialiser l'index et renommer la colonne
# df_buoy_datas = df_buoy_datas.reset_index(drop=False)
# df_buoy_datas.rename(columns={"index": "Station ID"}, inplace=True)

# table_name="buoy_datas"
# # Charger les données dans la table
# try:
#     load_data_in_table_sqlalchemy(conn=conn, schema=schema, df=df_buoy_datas, table_name=table_name, key_column="Station ID")
#     print(f"Données chargées avec succès dans la table {table_name}.")
# except Exception as e:
#     print(f"Erreur lors du chargement des données : {e}")

# # Affichage du DataFrame
# df_buoy_datas.head()

Map Visualization

In [12]:
# list_coords=[]

# for id in buoy_list:
#     metadata = get_station_metadata(id)
    
#     station_id, station_zone, lat_buoy, lon_buoy = parse_buoy_json(metadata)
#     lat_buoy, lon_buoy = convert_coordinates(lat_buoy,lon_buoy)
#     coords = [station_id, station_zone, lat_buoy, lon_buoy]
#     list_coords.append(coords)

# def barycentre(coords):
#     x_coords = [item[2] for item in coords]
#     y_coords = [item[3] for item in coords]

#     x_barycentre = sum(x_coords) / len(coords)
#     y_barycentre = sum(y_coords) / len(coords)

#     return [x_barycentre, y_barycentre]

# # Exemple d'utilisation
# coords = [[1, 2], [3, 4], [5, 6], [7, 8]]
# centre = barycentre(list_coords)
# print(list_coords)
# print("Coordonnées du barycentre :", centre)

In [13]:
# map = folium.Map(location=(centre[0],centre[1]), 
#                  tiles="Esri.WorldImagery", 
#                  zoom_start=2.5, attr="Données fournies par Esri")
# for loc in list_coords:
#     lat = loc[2]
#     lon = loc[3]
#     station_id = loc[0]
#     station_zone = loc[1]
#     folium.Marker(
#     location=[lat, lon],
#     popup=f"Station ID: {station_id}\n\nZone: {station_zone}\nLat: {lat}\n\nLon: {lon}",
#     icon=folium.Icon(icon="cloud"),
# ).add_to(map)

# map

Columns Check Tests

In [14]:
choice = random.choice(buoy_list)
choice

'46027'

In [15]:
df_marine_test = NDBC.realtime_observations(choice)
print(f'{df_marine_test.shape[0]} rows \n\n{df_marine_test.isna().sum()}')
df_marine_test.head(1)

6527 rows 

wind_direction               46
wind_speed                    5
wind_gust                     5
wave_height                3180
dominant_wave_period       4352
average_wave_period        3181
dominant_wave_direction    3181
pressure                      4
air_temperature               9
water_temperature            53
dewpoint                      9
visibility                 6527
3hr_pressure_tendency      5440
water_level_above_mean     6527
time                          0
dtype: int64


,wind_direction,wind_speed,wind_gust,wave_height,dominant_wave_period,average_wave_period,dominant_wave_direction,pressure,air_temperature,water_temperature,dewpoint,visibility,3hr_pressure_tendency,water_level_above_mean,time
0,350.0,6.0,8.0,NaN,NaN,NaN,NaN,1016.4,8.4,10.8,5.3,NaN,NaN,NaN,2025-04-02 12:30:00+00:00


Check test API Open-Meteo

In [16]:
df_meteo_test = meteo_api_request(coordinates=[12, 23])
print(f'{df_meteo_test.shape[0]} rows \n{df_meteo_test.isna().sum()}')
df_meteo_test.head(1)

2376 rows 
date                        0
temperature_2m            331
relative_humidity_2m      331
dew_point_2m              331
precipitation             331
rain                      331
showers                   331
pressure_msl              331
surface_pressure          331
cloud_cover               331
cloud_cover_low           331
cloud_cover_mid           331
cloud_cover_high          331
visibility                331
wind_speed_10m            331
soil_temperature_0cm      359
soil_moisture_0_to_1cm    359
is_day                      0
dtype: int64


,date,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,showers,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,wind_speed_10m,soil_temperature_0cm,soil_moisture_0_to_1cm,is_day
0,2024-12-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [17]:
metadata_extracted = get_station_metadata(choice)
metadata_extracted

{'Watch circle radius': '136 yards',
 'Water depth': '60 m',
 'Sea temp depth': '2 m below water line',
 'Barometer elevation': '2.4 m above mean sea level',
 'Anemometer height': '3.8 m above site elevation',
 'Air temp height': '3.4 m above site elevation',
 'Site elevation': 'sea level',
 'Location': '41.840 N 124.382 W (41°50\'24" N 124°22\'54" W)',
 'Statation Type': 'Owned and maintained by National Data Buoy Center, 3-meter discus buoy w/ seal cage, SCOOP payload',
 'Name': 'Station 46027 (LLNR 562) - ST GEORGES - 8 NM NW of Crescent City, CA'}

In [ ]:
# def drop_schema_or_tables(conn, schema_name, drop_schema=False):
#     try:
#         if drop_schema:
#             # Supprimer le schéma si drop_schema est True
#             print(f"Dropping schema {schema_name}...")
#             conn.execute(text(f"DROP SCHEMA IF EXISTS \"{schema_name}\" CASCADE;"))
#             conn.commit()
#             print(f"Schema {schema_name} dropped successfully.")
#         else:
#             # Supprimer toutes les tables dans le schéma
#             print(f"Dropping all tables in schema {schema_name}...")
#             conn.execute(text(f"""
#                 DO $$ 
#                 DECLARE
#                     r RECORD;
#                 BEGIN
#                     FOR r IN 
#                         SELECT table_name 
#                         FROM information_schema.tables 
#                         WHERE table_schema = :schema_name
#                     LOOP
#                         EXECUTE 'DROP TABLE IF EXISTS "' || :schema_name || '".' || quote_ident(r.table_name) || ' CASCADE';
#                     END LOOP;
#                 END $$;
#             """), {'schema_name': schema_name})
#             conn.commit()
#             print(f"All tables in schema {schema_name} dropped successfully.")
#     except Exception as e:
#         print(f"An error occurred: {e}")
#         conn.rollback()
#         print("Transaction rolled back.")

# drop_schema_or_tables(conn=conn, schema_name=schema, drop_schema=False)

In [ ]:
# def explore_dict(d, indent=0):
#     """ Fonction récursive pour afficher toute la structure du dictionnaire """
#     for key, value in d.items():
#         print(" " * indent + f"- {key}: {type(value)}")
#         if isinstance(value, dict):
#             explore_dict(value, indent + 4)  # Explorer récursivement avec une indentation
#         elif isinstance(value, list):
#             if len(value) > 0:
#                 print(" " * (indent + 4) + f"Liste ({len(value)} éléments), type du premier élément: {type(value[0])}")
#                 if isinstance(value[0], dict):
#                     explore_dict(value[0], indent + 8)  # Explorer si c'est une liste de dicts
#         else:
#             print(" " * (indent + 4) + f"Valeur: {value}")

# # Récupérer la première clé du dictionnaire
# first_key = next(iter(buoy_datas))
# print(f"Exploration de la première clé: {first_key}\n")

# # Exécuter la fonction sur le premier élément uniquement
# explore_dict({first_key: buoy_datas[first_key]})

In [ ]:
# openmongo_creds = r'C:\Users\f.gionnane\Documents\Data Engineering\Credentials\mongo_creds.json'

# with open(openmongo_creds, 'r') as file:
#     content = json.load(file)
#     mongo_user = content["user"]
#     mongo_password = content["password"]
#     mongo_string = content["connection_string"]

# uri = mongo_string
# # Create a new client and connect to the server
# client = MongoClient(uri, server_api=ServerApi('1'))

# # Send a ping to confirm a successful connection
# try:
#     client.admin.command('ping')
#     print("Pinged your deployment. You successfully connected to MongoDB!")
# except Exception as e:
#     print(e)
#     print(f'{mongo_user}\n{mongo_password}\n{mongo_string}')

In [ ]:
# Charger le fichier GeoJSON depuis l'URL
url = "https://gist.githubusercontent.com/jrrickard/8755532505a40f3b8317/raw/ecd98849d3a5f4502b773b986254f19af3b8d8fb/oceans.json"
geojson_data = requests.get(url).json()

# Créer une carte avec un fond satellite ESRI
m = folium.Map(location=[0, 0], zoom_start=3, tiles="Esri.WorldImagery")

# Ajouter le GeoJSON (les océans) à la carte
folium.GeoJson(geojson_data, name="Oceans").add_to(m)

# Fonction pour générer des coordonnées aléatoires dans l'océan
def random_ocean_coords():
    lat = random.uniform(-60, 60)  # Latitude dans les eaux de l'océan
    lon = random.uniform(-180, 180)  # Longitude dans les eaux de l'océan
    return lat, lon

# Générer un DataFrame avec des coordonnées, températures, hauteur des vagues, vitesse du vent, etc.
data = {
    "Coordinates": [],
    "Temperature (°C)": [],
    "Wave Height (m)": [],
    "Wind Speed (km/h)": []
}

# Générer 10 marqueurs avec des valeurs aléatoires
for _ in range(10):
    lat, lon = random_ocean_coords()
    
    # Générer des valeurs aléatoires pour la température, la hauteur des vagues et la vitesse du vent
    temperature = random.uniform(15, 30)  # Température entre 15 et 30°C
    wave_height = random.uniform(0.5, 5)  # Hauteur des vagues entre 0.5m et 5m
    wind_speed = random.uniform(10, 50)  # Vitesse du vent entre 10 km/h et 50 km/h
    
    # Ajouter ces valeurs dans le DataFrame
    data["Coordinates"].append((lat, lon))
    data["Temperature (°C)"].append(temperature)
    data["Wave Height (m)"].append(wave_height)
    data["Wind Speed (km/h)"].append(wind_speed)

# Créer un DataFrame pandas avec ces données
df = pd.DataFrame(data)

# Ajouter les marqueurs et cercles à la carte avec des couleurs dépendant de la température
for index, row in df.iterrows():
    lat, lon = row["Coordinates"]
    temperature = row["Temperature (°C)"]
    
    # Déterminer la couleur du cercle en fonction de la température
    if temperature < 20:
        fill_color = "blue"
    elif temperature < 25:
        fill_color = "green"
    else:
        fill_color = "red"
    
    # Ajouter un marqueur
    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(m)
    
    # Ajouter un cercle autour du marqueur
    folium.CircleMarker(
        location=[lat, lon],
        radius=15,  # Rayon du cercle
        color="black",
        fill=True,
        fill_color=fill_color,
        fill_opacity=0.5
    ).add_to(m)

# Ajouter une couche de contrôle pour la carte
folium.LayerControl().add_to(m)

# Sauvegarder la carte dans un fichier HTML
m.save("ocean_markers_map.html")

# Affichage de la carte dans l'environnement interactif (si vous êtes dans un environnement Jupyter)


In [ ]:
m